In [5]:
import numpy as np

In [6]:
class Dynamics:
    pass

In [7]:
hw2dynamics=Dynamics()
hw2dynamics.S0 = 100
hw2dynamics.r = 0.06
hw2dynamics.q = 0.01
hw2dynamics.maxvol = 0.6
hw2dynamics.localvol = lambda S,t: np.minimum(0.2+5*np.log(S/100)**2+0.1*np.exp(-t), 0.6)

# Note that hw2dynamics.localvol is a function 
# that may be invoked in the usual way, for example:
# hw2dynamics.localvol( exchangerate , time )

In [8]:
class Contract:
    pass

In [9]:
hw2contract=Contract()
hw2contract.putexpiry = 0.75;
hw2contract.putstrike = 95;
hw2contract.callexpiry = 0.25;
hw2contract.callstrike = 10;

In [10]:
class Tree:
    pass

In [11]:
hw2tree=Tree()
hw2tree.N=300  #change if necessary to get $0.01 accuracy, in your judgment

In [15]:
# You complete the coding of this function

def pricer_compound_localvol_trinom(contract,dynamics,tree):
    
    put_T, put_K, call_T, call_K = contract.putexpiry, contract.putstrike, contract.callexpiry, contract.callstrike
    S0, r, q, maxvol, localvol = dynamics.S0, dynamics.r, dynamics.q, dynamics.maxvol, dynamics.localvol
    N = tree.N
    
    deltat = put_T/N
    
    avgvol = localvol(S0, 0)
    deltax = np.maximum(avgvol*np.sqrt(3*deltat), maxvol*np.sqrt(deltat))
    
    S=S0*np.exp(np.linspace(N, -N, num=2*N+1, endpoint=True)*deltax)
    
    put_price = np.maximum(put_K-S,0) #Expiry time put prices
    
    for t in np.linspace(N-1, 0, num=N, endpoint=True)*deltat: ## induct backward
        S = S[1:-1]
        sigma = localvol(S,t) ## sigma is changing, so nu, pu, pd, pm are changing
        
        nu = r-q-sigma**2/2
        Pu = ((sigma**2*deltat+nu**2*deltat**2)/deltax**2+nu*deltat/deltax)/2
        Pd = ((sigma**2*deltat+nu**2*deltat**2)/deltax**2-nu*deltat/deltax)/2
        Pm = 1 - (sigma**2*deltat+nu**2*deltat**2)/(deltax**2)
        
        put_price = np.maximum(put_K-S, np.exp(-r*deltat)*(Pu*put_price[:-2]+Pm*put_price[1:-1]+Pd*put_price[2:]))
        
        if abs(t-call_T) < 1e-12: #Expiry time call prices. When t = 0.25
            call_price = np.maximum(put_price-call_K, 0)   
        
        if t-(call_T-deltat) < 1e-12: #Before expiry
            call_price = np.exp(-r*deltat)*(Pu*call_price[:-2]+Pm*call_price[1:-1]+Pd*call_price[2:])
            
    price_of_put =  put_price[0] #write code to compute this
    
    price_of_call_on_put = call_price[0] #write code to compute this
    
    return (price_of_put, price_of_call_on_put)

In [16]:
(answer_part_a, answer_part_b) = pricer_compound_localvol_trinom(hw2contract,hw2dynamics,hw2tree)

print(f'Answer Part (a) = {answer_part_a}')
print(f'Answer Part (b) = {answer_part_b}')

Answer Part (a) = 7.00588559542397
Answer Part (b) = 1.5937352664489162


In [17]:
## 2a

The exact formula for the Black-Scholes delta of a vanilla call is

\begin{equation}
\Delta = N(d_1),
\end{equation}

where

\begin{equation}
d_1 = \frac{\log(S / K) + (r + \frac{\sigma^2}{2}) (T - t)}{\sigma \sqrt{T - t}}.
\end{equation}

Plugging in $S = K$ and $t = 0$ yields

\begin{align}
d_1 &= \frac{(r + \frac{\sigma^2}{2}) T}{\sigma \sqrt{T}}\\
&= \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}.
\end{align}

Now, for $x \approx 0$, to first order,

\begin{equation}
N(x) \approx N(0) + N'(0) \cdot x.
\end{equation}

We know that $N(0) = 0.5$ and $N'(x) = \frac{1}{\sqrt{2 \pi}} \exp(\frac{-x^2}{2})$, so $N'(0) = \frac{1}{\sqrt{2\pi}} \approx 0.4$. Thus,

\begin{align}
\Delta &\approx 0.5 + \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma \sqrt{2 \pi}}\\
&\approx 0.5 + 0.4 \cdot \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}.
\end{align}

Plugging in $\sigma = 0.2$, $T = 0.25$, and $r = 0.01$ yields

\begin{align}
\Delta &\approx 0.5 + 0.4 \cdot \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}\\
&\approx 0.5 + 0.4 \cdot \frac{(0.01 + \frac{0.2^2}{2}) \sqrt{0.25}}{0.2}\\
&\approx 0.5 + 0.4 \cdot \frac{(0.01 + 0.02) 0.5}{0.2}\\
&\approx 0.5 + 0.4 \cdot \frac{0.03 \cdot 0.5}{0.2}\\
&\approx 0.5 + 0.4 \cdot \frac{0.015}{0.2}\\
&\approx 0.5 + 0.4 \cdot 0.075\\
&\approx 0.5 + 0.03\\
&\approx 0.53.
\end{align}

In [18]:
## 2b

Let $\Delta_\$$ and $\Gamma_\$$ denote the dollar delta and dollar gamma, respectively.

We are given $S_0 = 4$, $\Delta_\$ = 3$, $\Gamma_\$ = 0.02$, and $C(S_0) = 5$.

We find that

\begin{align}
\Delta_\$ = 3 &\implies S_0 \Delta = 3\\
&\implies 4 \Delta = 3\\
&\implies \Delta = \frac{3}{4}
\end{align}

and

\begin{align}
\Gamma_\$ = 0.02 &\implies \frac{1}{100} S_0^2 \Gamma = 0.02\\
&\implies S_0^2 \Gamma = 2\\
&\implies 4^2 \Gamma = 2\\
&\implies 16 \Gamma = 2\\
&\implies \Gamma = \frac{1}{8}.
\end{align}

Now, for $S \approx S_0$, to second order, we have

\begin{align}
C(S) &\approx C(S_0) + \Delta (S - S_0) + \frac{\Gamma}{2} (S - S_0)^2\\
&\approx 5 + \frac{3}{4} (S - 4) + \frac{1}{16} (S - 4)^2.
\end{align}

For $S = 3.6$, we have

\begin{align}
C(3.6) &\approx 5 + \frac{3}{4} (3.6 - 4) + \frac{1}{16} (3.6 - 4)^2\\
&\approx 5 + \frac{3}{4} (-0.4) + \frac{1}{16} (-0.4)^2\\
&\approx 5 - 0.3 + \frac{1}{16} \cdot 0.16\\
&\approx 5 - 0.3 + 0.01\\
&\approx 4.71.
\end{align}